In [ ]:
!pip install tqdm
!pip install torch
!pip install scikit_learn
!pip install nltk
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.1 MB/s eta 0:00:00


In [ ]:
import glob
import html
import os
from google.colab import drive

drive.mount('/content/drive')
BASE_PATH = '/content/drive/MyDrive'
DATA_PATH = os.path.join(BASE_PATH, "datasets")

SEPARATOR = "\t"


def clean_text(text):
    """
    Remove extra quotes from text files and html entities
    Args:
        text (str): a string of text

    Returns: (str): the "cleaned" text

    """
    text = text.rstrip()

    if '""' in text:
        if text[0] == text[-1] == '"':
            text = text[1:-1]
        text = text.replace('\\""', '"')
        text = text.replace('""', '"')

    text = text.replace('\\""', '"')

    text = html.unescape(text)
    text = ' '.join(text.split())
    return text


def parse_file(file):
    """
    Read a file and return a dictionary of the data, in the format:
    tweet_id:{sentiment, text}
    """

    data = {}
    lines = open(file, "r", encoding="utf-8").readlines()
    for _, line in enumerate(lines):
        columns = line.rstrip().split(SEPARATOR)
        tweet_id = columns[0]
        sentiment = columns[1]
        text = columns[2:]
        text = clean_text(" ".join(text))
        data[tweet_id] = (sentiment, text)
    return data


def load_from_dir(path):
    files = glob.glob(path + "/**/*.tsv", recursive=True)
    files.extend(glob.glob(path + "/**/*.txt", recursive=True))

    data = {}  # use dict, in order to avoid having duplicate tweets (same id)
    for file in files:
        file_data = parse_file(file)
        data.update(file_data)
    return list(data.values())


def load_Semeval2017A():
    train = load_from_dir(os.path.join(DATA_PATH, "Semeval2017A/train_dev"))
    test = load_from_dir(os.path.join(DATA_PATH, "Semeval2017A/gold"))

    X_train = [x[1] for x in train]
    y_train = [x[0] for x in train]
    X_test = [x[1] for x in test]
    y_test = [x[0] for x in test]

    return X_train, y_train, X_test, y_test


def load_MR():
    pos = open(os.path.join(DATA_PATH, "MR/rt-polarity.pos")).readlines()
    neg = open(os.path.join(DATA_PATH, "MR/rt-polarity.neg")).readlines()

    pos = [x.strip() for x in pos]
    neg = [x.strip() for x in neg]

    pos_labels = ["positive"] * len(pos)
    neg_labels = ["negative"] * len(neg)

    split = 5000

    X_train = pos[:split] + neg[:split]
    y_train = pos_labels[:split] + neg_labels[:split]

    X_test = pos[split:] + neg[split:]
    y_test = pos_labels[split:] + neg_labels[split:]

    return X_train, y_train, X_test, y_test

Mounted at /content/drive


In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:32

In [ ]:
import numpy as np
import evaluate
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
torch.cuda.empty_cache()
PRETRAINED_MODEL_MR = ['bert-base-cased', 'bert-base-uncased', 'distilbert-base-uncased', 'distilroberta-base']
PRETRAINED_MODEL_SEM = ['distilbert-base-cased', 'distilbert-base-uncased', 'albert-base-v2', 'bert-base-uncased']

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def prepare_dataset(X, y):
    texts, labels = [], []
    for text, label in zip(X, y):
        texts.append(text)
        labels.append(label)

    return Dataset.from_dict({'text': texts, 'label': labels})


if __name__ == '__main__':

      # load the raw data
      DATASET = 'MR'
      PRETRAINED_MODEL = 'bert-base-cased'
      if DATASET == "Semeval2017A":
            X_train, y_train, X_test, y_test = load_Semeval2017A()
      elif DATASET == "MR":
            X_train, y_train, X_test, y_test = load_MR()
      else:
          raise ValueError("Invalid dataset")

      # encode labels
      le = LabelEncoder()
      le.fit(list(set(y_train)))
      y_train = le.transform(y_train)
      y_test = le.transform(y_test)
      n_classes = len(list(le.classes_))

      # prepare datasets
      train_set = prepare_dataset(X_train, y_train)
      test_set = prepare_dataset(X_test, y_test)

      # define model and tokenizer
      tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
      model = AutoModelForSequenceClassification.from_pretrained(
          PRETRAINED_MODEL, num_labels=n_classes)

      # tokenize datasets
      tokenized_train_set = train_set.map(tokenize_function)
      tokenized_test_set = test_set.map(tokenize_function)

      # TODO: Main-lab-Q7 - remove this section once you are ready to execute on a GPU
      #  create a smaller subset of the dataset
      n_samples = 400
      small_train_dataset = tokenized_train_set.shuffle(
          seed=42).select(range(n_samples))
      small_eval_dataset = tokenized_test_set.shuffle(
          seed=42).select(range(n_samples))

      # TODO: Main-lab-Q7 - customize hyperparameters once you are ready to execute on a GPU
      # training setup
      print(f'\nDataset: {DATASET}\nPre-Trained model: {PRETRAINED_MODEL}\n')

      args = TrainingArguments(
        output_dir="output",
        evaluation_strategy="epoch",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
      )
      trainer = Trainer(
          model=model,
          args=args,
          train_dataset=small_train_dataset,
          eval_dataset=small_eval_dataset,
          compute_metrics=compute_metrics,
      )

      # train
      trained_model = trainer.train()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]


Dataset: MR
Pre-Trained model: bert-base-cased



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.516899,0.752500
2,No log,1.266807,0.775000
3,No log,1.368540,0.785000
4,No log,1.356325,0.812500
5,0.233400,1.531102,0.802500
6,0.233400,1.533484,0.802500
7,0.233400,1.563491,0.800000
8,0.233400,1.604665,0.807500
9,0.233400,1.621222,0.807500
10,0.000900,1.628371,0.807500


In [ ]:
import numpy as np
import evaluate
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
torch.cuda.empty_cache()

##############
PRETRAINED_MODEL_MR = ['bert-base-cased', 'bert-base-uncased', 'distilbert-base-uncased', 'distilroberta-base']
PRETRAINED_MODEL_SEM = ['distilbert-base-cased', 'distilbert-base-uncased', 'albert-base-v2', 'bert-base-uncased']

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def prepare_dataset(X, y):
    texts, labels = [], []
    for text, label in zip(X, y):
        texts.append(text)
        labels.append(label)

    return Dataset.from_dict({'text': texts, 'label': labels})


if __name__ == '__main__':

      # load the raw data
      DATASET = 'MR'
      PRETRAINED_MODEL = 'bert-base-uncased'
      if DATASET == "Semeval2017A":
            X_train, y_train, X_test, y_test = load_Semeval2017A()
      elif DATASET == "MR":
            X_train, y_train, X_test, y_test = load_MR()
      else:
          raise ValueError("Invalid dataset")

      # encode labels
      le = LabelEncoder()
      le.fit(list(set(y_train)))
      y_train = le.transform(y_train)
      y_test = le.transform(y_test)
      n_classes = len(list(le.classes_))

      # prepare datasets
      train_set = prepare_dataset(X_train, y_train)
      test_set = prepare_dataset(X_test, y_test)

      # define model and tokenizer
      tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
      model = AutoModelForSequenceClassification.from_pretrained(
          PRETRAINED_MODEL, num_labels=n_classes)

      # tokenize datasets
      tokenized_train_set = train_set.map(tokenize_function)
      tokenized_test_set = test_set.map(tokenize_function)

      # TODO: Main-lab-Q7 - remove this section once you are ready to execute on a GPU
      #  create a smaller subset of the dataset
      n_samples = 400
      small_train_dataset = tokenized_train_set.shuffle(
          seed=42).select(range(n_samples))
      small_eval_dataset = tokenized_test_set.shuffle(
          seed=42).select(range(n_samples))

      # TODO: Main-lab-Q7 - customize hyperparameters once you are ready to execute on a GPU
      # training setup
      print(f'\nDataset: {DATASET}\nPre-Trained model: {PRETRAINED_MODEL}\n')

      args = TrainingArguments(
        output_dir="output",
        evaluation_strategy="epoch",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
      )
      trainer = Trainer(
          model=model,
          args=args,
          train_dataset=small_train_dataset,
          eval_dataset=small_eval_dataset,
          compute_metrics=compute_metrics,
      )

      # train
      trained_model = trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]


Dataset: MR
Pre-Trained model: bert-base-uncased



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.802864,0.767500
2,No log,1.370706,0.772500
3,No log,1.089064,0.815000
4,No log,1.146359,0.837500
5,0.212400,1.191740,0.842500
6,0.212400,1.230707,0.845000
7,0.212400,1.257243,0.842500
8,0.212400,1.274726,0.845000
9,0.212400,1.285037,0.845000
10,0.000100,1.288708,0.845000


In [ ]:
import numpy as np
import evaluate
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
torch.cuda.empty_cache()
PRETRAINED_MODEL_MR = ['bert-base-cased', 'bert-base-uncased', 'distilbert-base-uncased', 'distilroberta-base']
PRETRAINED_MODEL_SEM = ['distilbert-base-cased', 'distilbert-base-uncased', 'albert-base-v2', 'bert-base-uncased']

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def prepare_dataset(X, y):
    texts, labels = [], []
    for text, label in zip(X, y):
        texts.append(text)
        labels.append(label)

    return Dataset.from_dict({'text': texts, 'label': labels})


if __name__ == '__main__':

      # load the raw data
      DATASET = 'MR'
      PRETRAINED_MODEL = 'distilbert-base-uncased'
      if DATASET == "Semeval2017A":
            X_train, y_train, X_test, y_test = load_Semeval2017A()
      elif DATASET == "MR":
            X_train, y_train, X_test, y_test = load_MR()
      else:
          raise ValueError("Invalid dataset")

      # encode labels
      le = LabelEncoder()
      le.fit(list(set(y_train)))
      y_train = le.transform(y_train)
      y_test = le.transform(y_test)
      n_classes = len(list(le.classes_))

      # prepare datasets
      train_set = prepare_dataset(X_train, y_train)
      test_set = prepare_dataset(X_test, y_test)

      # define model and tokenizer
      tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
      model = AutoModelForSequenceClassification.from_pretrained(
          PRETRAINED_MODEL, num_labels=n_classes)

      # tokenize datasets
      tokenized_train_set = train_set.map(tokenize_function)
      tokenized_test_set = test_set.map(tokenize_function)

      # TODO: Main-lab-Q7 - remove this section once you are ready to execute on a GPU
      #  create a smaller subset of the dataset
      n_samples = 400
      small_train_dataset = tokenized_train_set.shuffle(
          seed=42).select(range(n_samples))
      small_eval_dataset = tokenized_test_set.shuffle(
          seed=42).select(range(n_samples))

      # TODO: Main-lab-Q7 - customize hyperparameters once you are ready to execute on a GPU
      # training setup
      print(f'\nDataset: {DATASET}\nPre-Trained model: {PRETRAINED_MODEL}\n')

      args = TrainingArguments(
        output_dir="output",
        evaluation_strategy="epoch",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
      )
      trainer = Trainer(
          model=model,
          args=args,
          train_dataset=small_train_dataset,
          eval_dataset=small_eval_dataset,
          compute_metrics=compute_metrics,
      )

      # train
      trained_model = trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.we

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]


Dataset: MR
Pre-Trained model: distilbert-base-uncased



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.601335,0.720000
2,No log,1.172449,0.735000
3,No log,1.169616,0.775000
4,No log,1.233487,0.807500
5,0.225600,1.372808,0.797500
6,0.225600,1.453992,0.792500
7,0.225600,1.514231,0.790000
8,0.225600,1.544617,0.790000
9,0.225600,1.543927,0.785000
10,0.000200,1.552416,0.785000


In [ ]:
import numpy as np
import evaluate
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
torch.cuda.empty_cache()
PRETRAINED_MODEL_MR = ['bert-base-cased', 'bert-base-uncased', 'distilbert-base-uncased', 'distilroberta-base']
PRETRAINED_MODEL_SEM = ['distilbert-base-cased', 'distilbert-base-uncased', 'albert-base-v2', 'bert-base-uncased']

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def prepare_dataset(X, y):
    texts, labels = [], []
    for text, label in zip(X, y):
        texts.append(text)
        labels.append(label)

    return Dataset.from_dict({'text': texts, 'label': labels})


if __name__ == '__main__':

      # load the raw data
      DATASET = 'MR'
      PRETRAINED_MODEL = 'distilroberta-base'
      if DATASET == "Semeval2017A":
            X_train, y_train, X_test, y_test = load_Semeval2017A()
      elif DATASET == "MR":
            X_train, y_train, X_test, y_test = load_MR()
      else:
          raise ValueError("Invalid dataset")

      # encode labels
      le = LabelEncoder()
      le.fit(list(set(y_train)))
      y_train = le.transform(y_train)
      y_test = le.transform(y_test)
      n_classes = len(list(le.classes_))

      # prepare datasets
      train_set = prepare_dataset(X_train, y_train)
      test_set = prepare_dataset(X_test, y_test)

      # define model and tokenizer
      tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
      model = AutoModelForSequenceClassification.from_pretrained(
          PRETRAINED_MODEL, num_labels=n_classes)

      # tokenize datasets
      tokenized_train_set = train_set.map(tokenize_function)
      tokenized_test_set = test_set.map(tokenize_function)

      # TODO: Main-lab-Q7 - remove this section once you are ready to execute on a GPU
      #  create a smaller subset of the dataset
      n_samples = 400
      small_train_dataset = tokenized_train_set.shuffle(
          seed=42).select(range(n_samples))
      small_eval_dataset = tokenized_test_set.shuffle(
          seed=42).select(range(n_samples))

      # TODO: Main-lab-Q7 - customize hyperparameters once you are ready to execute on a GPU
      # training setup
      print(f'\nDataset: {DATASET}\nPre-Trained model: {PRETRAINED_MODEL}\n')

      args = TrainingArguments(
        output_dir="output",
        evaluation_strategy="epoch",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
      )
      trainer = Trainer(
          model=model,
          args=args,
          train_dataset=small_train_dataset,
          eval_dataset=small_eval_dataset,
          compute_metrics=compute_metrics,
      )

      # train
      trained_model = trainer.train()

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bia

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]


Dataset: MR
Pre-Trained model: distilroberta-base



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.633308,0.675000
2,No log,0.870643,0.762500
3,No log,1.325891,0.755000
4,No log,1.291804,0.777500
5,0.342200,1.625464,0.772500
6,0.342200,1.512927,0.787500
7,0.342200,1.546722,0.800000
8,0.342200,1.626568,0.795000
9,0.342200,1.629778,0.797500
10,0.010200,1.637603,0.800000


In [ ]:
import numpy as np
import evaluate
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
torch.cuda.empty_cache()
PRETRAINED_MODEL_MR = ['bert-base-cased', 'bert-base-uncased', 'distilbert-base-uncased', 'distilroberta-base']
PRETRAINED_MODEL_SEM = ['distilbert-base-cased', 'distilbert-base-uncased', 'albert-base-v2', 'bert-base-uncased']

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def prepare_dataset(X, y):
    texts, labels = [], []
    for text, label in zip(X, y):
        texts.append(text)
        labels.append(label)

    return Dataset.from_dict({'text': texts, 'label': labels})


if __name__ == '__main__':

      # load the raw data
      DATASET = 'Semeval2017A'
      PRETRAINED_MODEL = 'distilbert-base-cased'
      if DATASET == "Semeval2017A":
            X_train, y_train, X_test, y_test = load_Semeval2017A()
      elif DATASET == "MR":
            X_train, y_train, X_test, y_test = load_MR()
      else:
          raise ValueError("Invalid dataset")

      # encode labels
      le = LabelEncoder()
      le.fit(list(set(y_train)))
      y_train = le.transform(y_train)
      y_test = le.transform(y_test)
      n_classes = len(list(le.classes_))

      # prepare datasets
      train_set = prepare_dataset(X_train, y_train)
      test_set = prepare_dataset(X_test, y_test)

      # define model and tokenizer
      tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
      model = AutoModelForSequenceClassification.from_pretrained(
          PRETRAINED_MODEL, num_labels=n_classes)

      # tokenize datasets
      tokenized_train_set = train_set.map(tokenize_function)
      tokenized_test_set = test_set.map(tokenize_function)

      # TODO: Main-lab-Q7 - remove this section once you are ready to execute on a GPU
      #  create a smaller subset of the dataset
      n_samples = 400
      small_train_dataset = tokenized_train_set.shuffle(
          seed=42).select(range(n_samples))
      small_eval_dataset = tokenized_test_set.shuffle(
          seed=42).select(range(n_samples))

      # TODO: Main-lab-Q7 - customize hyperparameters once you are ready to execute on a GPU
      # training setup
      print(f'\nDataset: {DATASET}\nPre-Trained model: {PRETRAINED_MODEL}\n')

      args = TrainingArguments(
        output_dir="output",
        evaluation_strategy="epoch",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
      )
      trainer = Trainer(
          model=model,
          args=args,
          train_dataset=small_train_dataset,
          eval_dataset=small_eval_dataset,
          compute_metrics=compute_metrics,
      )

      # train
      trained_model = trainer.train()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.w

Map:   0%|          | 0/49570 [00:00<?, ? examples/s]

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]


Dataset: Semeval2017A
Pre-Trained model: distilbert-base-cased



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.994051,0.465000
2,No log,0.961171,0.570000
3,No log,1.656690,0.585000
4,No log,1.891026,0.572500
5,0.505900,2.287052,0.580000
6,0.505900,2.376569,0.580000
7,0.505900,2.573236,0.577500
8,0.505900,2.751299,0.572500
9,0.505900,2.771646,0.572500
10,0.001100,2.778908,0.575000


In [ ]:
import numpy as np
import evaluate
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
torch.cuda.empty_cache()
PRETRAINED_MODEL_MR = ['bert-base-cased', 'bert-base-uncased', 'distilbert-base-uncased', 'distilroberta-base']
PRETRAINED_MODEL_SEM = ['distilbert-base-cased', 'distilbert-base-uncased', 'albert-base-v2', 'bert-base-uncased']


metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def prepare_dataset(X, y):
    texts, labels = [], []
    for text, label in zip(X, y):
        texts.append(text)
        labels.append(label)

    return Dataset.from_dict({'text': texts, 'label': labels})


if __name__ == '__main__':

      # load the raw data
      DATASET = 'Semeval2017A'
      PRETRAINED_MODEL = 'distilbert-base-uncased'
      if DATASET == "Semeval2017A":
            X_train, y_train, X_test, y_test = load_Semeval2017A()
      elif DATASET == "MR":
            X_train, y_train, X_test, y_test = load_MR()
      else:
          raise ValueError("Invalid dataset")

      # encode labels
      le = LabelEncoder()
      le.fit(list(set(y_train)))
      y_train = le.transform(y_train)
      y_test = le.transform(y_test)
      n_classes = len(list(le.classes_))

      # prepare datasets
      train_set = prepare_dataset(X_train, y_train)
      test_set = prepare_dataset(X_test, y_test)

      # define model and tokenizer
      tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
      model = AutoModelForSequenceClassification.from_pretrained(
          PRETRAINED_MODEL, num_labels=n_classes)

      # tokenize datasets
      tokenized_train_set = train_set.map(tokenize_function)
      tokenized_test_set = test_set.map(tokenize_function)

      # TODO: Main-lab-Q7 - remove this section once you are ready to execute on a GPU
      #  create a smaller subset of the dataset
      n_samples = 400
      small_train_dataset = tokenized_train_set.shuffle(
          seed=42).select(range(n_samples))
      small_eval_dataset = tokenized_test_set.shuffle(
          seed=42).select(range(n_samples))

      # TODO: Main-lab-Q7 - customize hyperparameters once you are ready to execute on a GPU
      # training setup
      print(f'\nDataset: {DATASET}\nPre-Trained model: {PRETRAINED_MODEL}\n')

      args = TrainingArguments(
        output_dir="output",
        evaluation_strategy="epoch",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
      )
      trainer = Trainer(
          model=model,
          args=args,
          train_dataset=small_train_dataset,
          eval_dataset=small_eval_dataset,
          compute_metrics=compute_metrics,
      )

      # train
      trained_model = trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.we

Map:   0%|          | 0/49570 [00:00<?, ? examples/s]

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]


Dataset: Semeval2017A
Pre-Trained model: distilbert-base-uncased



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.965626,0.465000
2,No log,0.852479,0.607500
3,No log,1.572780,0.607500
4,No log,2.018121,0.615000
5,0.442300,1.984584,0.630000
6,0.442300,2.287934,0.595000
7,0.442300,2.465963,0.597500
8,0.442300,2.395487,0.592500
9,0.442300,2.399767,0.605000
10,0.000900,2.411487,0.605000


In [ ]:
import numpy as np
import evaluate
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
torch.cuda.empty_cache()
PRETRAINED_MODEL_MR = ['bert-base-cased', 'bert-base-uncased', 'distilbert-base-uncased', 'distilroberta-base']
PRETRAINED_MODEL_SEM = ['distilbert-base-cased', 'distilbert-base-uncased', 'albert-base-v2', 'bert-base-uncased']


metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def prepare_dataset(X, y):
    texts, labels = [], []
    for text, label in zip(X, y):
        texts.append(text)
        labels.append(label)

    return Dataset.from_dict({'text': texts, 'label': labels})


if __name__ == '__main__':

      # load the raw data
      DATASET = 'Semeval2017A'
      PRETRAINED_MODEL = 'albert-base-v2'
      if DATASET == "Semeval2017A":
            X_train, y_train, X_test, y_test = load_Semeval2017A()
      elif DATASET == "MR":
            X_train, y_train, X_test, y_test = load_MR()
      else:
          raise ValueError("Invalid dataset")

      # encode labels
      le = LabelEncoder()
      le.fit(list(set(y_train)))
      y_train = le.transform(y_train)
      y_test = le.transform(y_test)
      n_classes = len(list(le.classes_))

      # prepare datasets
      train_set = prepare_dataset(X_train, y_train)
      test_set = prepare_dataset(X_test, y_test)

      # define model and tokenizer
      tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
      model = AutoModelForSequenceClassification.from_pretrained(
          PRETRAINED_MODEL, num_labels=n_classes)

      # tokenize datasets
      tokenized_train_set = train_set.map(tokenize_function)
      tokenized_test_set = test_set.map(tokenize_function)

      # TODO: Main-lab-Q7 - remove this section once you are ready to execute on a GPU
      #  create a smaller subset of the dataset
      n_samples = 400
      small_train_dataset = tokenized_train_set.shuffle(
          seed=42).select(range(n_samples))
      small_eval_dataset = tokenized_test_set.shuffle(
          seed=42).select(range(n_samples))

      # TODO: Main-lab-Q7 - customize hyperparameters once you are ready to execute on a GPU
      # training setup
      print(f'\nDataset: {DATASET}\nPre-Trained model: {PRETRAINED_MODEL}\n')

      args = TrainingArguments(
        output_dir="output",
        evaluation_strategy="epoch",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
      )
      trainer = Trainer(
          model=model,
          args=args,
          train_dataset=small_train_dataset,
          eval_dataset=small_eval_dataset,
          compute_metrics=compute_metrics,
      )

      # train
      trained_model = trainer.train()

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model 

Map:   0%|          | 0/49570 [00:00<?, ? examples/s]

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]


Dataset: Semeval2017A
Pre-Trained model: albert-base-v2



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.056818,0.480000
2,No log,1.180015,0.460000
3,No log,1.154089,0.460000
4,No log,1.121303,0.432500
5,1.095800,1.171339,0.460000
6,1.095800,1.091835,0.460000
7,1.095800,1.160875,0.460000
8,1.095800,1.100862,0.460000
9,1.095800,1.124585,0.460000
10,1.081600,1.130775,0.460000


In [ ]:
import numpy as np
import evaluate
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
torch.cuda.empty_cache()
PRETRAINED_MODEL_MR = ['bert-base-cased', 'bert-base-uncased', 'distilbert-base-uncased', 'distilroberta-base']
PRETRAINED_MODEL_SEM = ['distilbert-base-cased', 'distilbert-base-uncased', 'albert-base-v2', 'bert-base-uncased']


metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def prepare_dataset(X, y):
    texts, labels = [], []
    for text, label in zip(X, y):
        texts.append(text)
        labels.append(label)

    return Dataset.from_dict({'text': texts, 'label': labels})


if __name__ == '__main__':

      # load the raw data
      DATASET = 'Semeval2017A'
      PRETRAINED_MODEL = 'bert-base-uncased'
      if DATASET == "Semeval2017A":
            X_train, y_train, X_test, y_test = load_Semeval2017A()
      elif DATASET == "MR":
            X_train, y_train, X_test, y_test = load_MR()
      else:
          raise ValueError("Invalid dataset")

      # encode labels
      le = LabelEncoder()
      le.fit(list(set(y_train)))
      y_train = le.transform(y_train)
      y_test = le.transform(y_test)
      n_classes = len(list(le.classes_))

      # prepare datasets
      train_set = prepare_dataset(X_train, y_train)
      test_set = prepare_dataset(X_test, y_test)

      # define model and tokenizer
      tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
      model = AutoModelForSequenceClassification.from_pretrained(
          PRETRAINED_MODEL, num_labels=n_classes)

      # tokenize datasets
      tokenized_train_set = train_set.map(tokenize_function)
      tokenized_test_set = test_set.map(tokenize_function)

      # TODO: Main-lab-Q7 - remove this section once you are ready to execute on a GPU
      #  create a smaller subset of the dataset
      n_samples = 400
      small_train_dataset = tokenized_train_set.shuffle(
          seed=42).select(range(n_samples))
      small_eval_dataset = tokenized_test_set.shuffle(
          seed=42).select(range(n_samples))

      # TODO: Main-lab-Q7 - customize hyperparameters once you are ready to execute on a GPU
      # training setup
      print(f'\nDataset: {DATASET}\nPre-Trained model: {PRETRAINED_MODEL}\n')

      args = TrainingArguments(
        output_dir="output",
        evaluation_strategy="epoch",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
      )
      trainer = Trainer(
          model=model,
          args=args,
          train_dataset=small_train_dataset,
          eval_dataset=small_eval_dataset,
          compute_metrics=compute_metrics,
      )

      # train
      trained_model = trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Map:   0%|          | 0/49570 [00:00<?, ? examples/s]

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]


Dataset: Semeval2017A
Pre-Trained model: bert-base-uncased



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.056626,0.460000
2,No log,0.906902,0.595000
3,No log,1.037395,0.622500
4,No log,1.890174,0.582500
5,0.714000,1.730373,0.622500
6,0.714000,1.922395,0.620000
7,0.714000,2.241256,0.600000
8,0.714000,2.357258,0.612500
9,0.714000,2.380605,0.602500
10,0.066400,2.403538,0.602500
